# ਆਟੋਜਨ ਬੇਸਿਕ ਸੈਂਪਲ

ਇਸ ਕੋਡ ਸੈਂਪਲ ਵਿੱਚ, ਤੁਸੀਂ [AutoGen](https://aka.ms/ai-agents/autogen) AI ਫਰੇਮਵਰਕ ਦੀ ਵਰਤੋਂ ਕਰਕੇ ਇੱਕ ਬੇਸਿਕ ਏਜੰਟ ਬਣਾਉਗੇ।

ਇਸ ਸੈਂਪਲ ਦਾ ਮਕਸਦ ਤੁਹਾਨੂੰ ਉਹ ਕਦਮ ਦਿਖਾਉਣਾ ਹੈ ਜੋ ਅਸੀਂ ਵੱਖ-ਵੱਖ ਏਜੰਟਿਕ ਪੈਟਰਨ ਨੂੰ ਲਾਗੂ ਕਰਦੇ ਸਮੇਂ ਅਗਲੇ ਕੋਡ ਸੈਂਪਲ ਵਿੱਚ ਵਰਤਾਂਗੇ।


## ਲੋੜੀਂਦੇ ਪਾਇਥਨ ਪੈਕੇਜਾਂ ਨੂੰ ਇੰਪੋਰਟ ਕਰੋ


In [ ]:
import os
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken

from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console


## ਕਲਾਇੰਟ ਬਣਾਓ

ਇਸ ਉਦਾਹਰਨ ਵਿੱਚ, ਅਸੀਂ [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) ਦੀ ਵਰਤੋਂ ਕਰਾਂਗੇ LLM ਤੱਕ ਪਹੁੰਚ ਲਈ।

`model` ਨੂੰ `gpt-4o-mini` ਵਜੋਂ ਪਰਿਭਾਸ਼ਿਤ ਕੀਤਾ ਗਿਆ ਹੈ। GitHub Models ਮਾਰਕੀਟਪਲੇਸ ਵਿੱਚ ਉਪਲਬਧ ਕਿਸੇ ਹੋਰ ਮਾਡਲ ਨਾਲ ਮਾਡਲ ਬਦਲਣ ਦੀ ਕੋਸ਼ਿਸ਼ ਕਰੋ ਤਾਂ ਜੋ ਵੱਖ-ਵੱਖ ਨਤੀਜੇ ਵੇਖੇ ਜਾ ਸਕਣ।

ਇੱਕ ਤੇਜ਼ ਟੈਸਟ ਵਜੋਂ, ਅਸੀਂ ਸਿਰਫ ਇੱਕ ਸਧਾਰਨ ਪ੍ਰੰਪਟ ਚਲਾਵਾਂਗੇ - `ਫਰਾਂਸ ਦੀ ਰਾਜਧਾਨੀ ਕੀ ਹੈ`।


In [ ]:
load_dotenv()
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## ਏਜੰਟ ਦੀ ਪਰਿਭਾਸ਼ਾ

ਹੁਣ ਜਦੋਂ ਅਸੀਂ `client` ਸੈਟਅੱਪ ਕਰ ਲਿਆ ਹੈ ਅਤੇ ਇਸਦੀ ਕਾਰਗੁਜ਼ਾਰੀ ਦੀ ਪੁਸ਼ਟੀ ਕਰ ਲਈ ਹੈ, ਆਓ ਇੱਕ `AssistantAgent` ਬਣਾਈਏ। ਹਰ ਏਜੰਟ ਨੂੰ ਇਹ ਦਿੱਤਾ ਜਾ ਸਕਦਾ ਹੈ:  
**name** - ਇੱਕ ਛੋਟਾ ਨਾਮ ਜੋ ਇਸਨੂੰ ਬਹੁ-ਏਜੰਟ ਪ੍ਰਕਿਰਿਆਵਾਂ ਵਿੱਚ ਰਿਫਰ ਕਰਨ ਲਈ ਸਹਾਇਕ ਹੋਵੇਗਾ।  
**model_client** - ਉਹ ਕਲਾਇੰਟ ਜੋ ਤੁਸੀਂ ਪਹਿਲੇ ਕਦਮ ਵਿੱਚ ਬਣਾਇਆ ਸੀ।  
**tools** - ਉਹ ਸਾਧਨ ਜੋ ਏਜੰਟ ਕਿਸੇ ਕੰਮ ਨੂੰ ਪੂਰਾ ਕਰਨ ਲਈ ਵਰਤ ਸਕਦਾ ਹੈ।  
**system_message** - ਮੈਟਾਪ੍ਰਾਂਪਟ ਜੋ LLM ਦੇ ਕੰਮ, ਵਿਵਹਾਰ ਅਤੇ ਲਹਿਜ਼ੇ ਨੂੰ ਪਰਿਭਾਸ਼ਿਤ ਕਰਦਾ ਹੈ।  

ਤੁਸੀਂ ਸਿਸਟਮ ਸੁਨੇਹਾ ਬਦਲ ਸਕਦੇ ਹੋ ਤਾਂ ਜੋ ਵੇਖ ਸਕੋ ਕਿ LLM ਕਿਵੇਂ ਪ੍ਰਤੀਕਿਰਿਆ ਕਰਦਾ ਹੈ। ਅਸੀਂ `tools` ਨੂੰ ਪਾਠ #4 ਵਿੱਚ ਕਵਰ ਕਰਾਂਗੇ।  


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="You are a travel agent that plans great vacations",
)

## ਏਜੰਟ ਚਲਾਓ

ਹੇਠਾਂ ਦਿੱਤੀ ਫੰਕਸ਼ਨ ਏਜੰਟ ਨੂੰ ਚਲਾਏਗੀ। ਅਸੀਂ `on_message` ਵਿਧੀ ਦੀ ਵਰਤੋਂ ਕਰਦੇ ਹਾਂ ਤਾਂ ਜੋ ਨਵੇਂ ਸੁਨੇਹੇ ਨਾਲ ਏਜੰਟ ਦੀ ਸਥਿਤੀ ਨੂੰ ਅਪਡੇਟ ਕੀਤਾ ਜਾ ਸਕੇ।

ਇਸ ਮਾਮਲੇ ਵਿੱਚ, ਅਸੀਂ ਯੂਜ਼ਰ ਵੱਲੋਂ ਆਏ ਨਵੇਂ ਸੁਨੇਹੇ ਨਾਲ ਸਥਿਤੀ ਨੂੰ ਅਪਡੇਟ ਕਰਦੇ ਹਾਂ, ਜੋ ਕਿ ਹੈ `"Plan me a great sunny vacation"`।

ਤੁਸੀਂ ਸੁਨੇਹੇ ਦੀ ਸਮੱਗਰੀ ਨੂੰ ਬਦਲ ਸਕਦੇ ਹੋ ਤਾਂ ਜੋ ਵੇਖ ਸਕੋ ਕਿ LLM ਕਿਵੇਂ ਵੱਖ-ਵੱਖ ਤਰੀਕਿਆਂ ਨਾਲ ਜਵਾਬ ਦਿੰਦਾ ਹੈ।


In [ ]:
from IPython.display import display, HTML


async def assistant_run():
    # Define the query
    user_query = "Plan me a great sunny vacation"

    # Start building HTML output
    html_output = "<div style='margin-bottom:10px'>"
    html_output += "<div style='font-weight:bold'>User:</div>"
    html_output += f"<div style='margin-left:20px'>{user_query}</div>"
    html_output += "</div>"

    # Execute the agent response
    response = await agent.on_messages(
        [TextMessage(content=user_query, source="user")],
        cancellation_token=CancellationToken(),
    )

    # Add agent response to HTML
    html_output += "<div style='margin-bottom:20px'>"
    html_output += "<div style='font-weight:bold'>Assistant:</div>"
    html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{response.chat_message.content}</div>"
    html_output += "</div>"

    # Display formatted HTML
    display(HTML(html_output))

# Run the function
await assistant_run()


---

**ਅਸਵੀਕਰਤੀ**:  
ਇਹ ਦਸਤਾਵੇਜ਼ AI ਅਨੁਵਾਦ ਸੇਵਾ [Co-op Translator](https://github.com/Azure/co-op-translator) ਦੀ ਵਰਤੋਂ ਕਰਕੇ ਅਨੁਵਾਦ ਕੀਤਾ ਗਿਆ ਹੈ। ਜਦੋਂ ਕਿ ਅਸੀਂ ਸਹੀ ਹੋਣ ਦਾ ਯਤਨ ਕਰਦੇ ਹਾਂ, ਕਿਰਪਾ ਕਰਕੇ ਧਿਆਨ ਦਿਓ ਕਿ ਸਵੈਚਾਲਿਤ ਅਨੁਵਾਦਾਂ ਵਿੱਚ ਗਲਤੀਆਂ ਜਾਂ ਅਸੁੱਤੀਆਂ ਹੋ ਸਕਦੀਆਂ ਹਨ। ਇਸ ਦੀ ਮੂਲ ਭਾਸ਼ਾ ਵਿੱਚ ਮੌਜੂਦ ਮੂਲ ਦਸਤਾਵੇਜ਼ ਨੂੰ ਅਧਿਕਾਰਤ ਸਰੋਤ ਮੰਨਿਆ ਜਾਣਾ ਚਾਹੀਦਾ ਹੈ। ਮਹੱਤਵਪੂਰਨ ਜਾਣਕਾਰੀ ਲਈ, ਪੇਸ਼ੇਵਰ ਮਨੁੱਖੀ ਅਨੁਵਾਦ ਦੀ ਸਿਫਾਰਸ਼ ਕੀਤੀ ਜਾਂਦੀ ਹੈ। ਇਸ ਅਨੁਵਾਦ ਦੇ ਪ੍ਰਯੋਗ ਤੋਂ ਪੈਦਾ ਹੋਣ ਵਾਲੇ ਕਿਸੇ ਵੀ ਗਲਤਫਹਮੀਆਂ ਜਾਂ ਗਲਤ ਵਿਆਖਿਆਵਾਂ ਲਈ ਅਸੀਂ ਜ਼ਿੰਮੇਵਾਰ ਨਹੀਂ ਹਾਂ।  
